In [18]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',300)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [19]:
target = pd.read_csv('target2',header=None,index_col=0)

In [20]:
train = feather.read_dataframe('best_from_recent_groupped_train')
test = feather.read_dataframe('best_from_recent_groupped_test')

In [21]:
new_test_cols = feather.read_dataframe('new_test_columns_with_promos_fixed')

train = pd.merge(train,new_test_cols,on='card_id',how='left')
test = pd.merge(test,new_test_cols,on='card_id',how='left')

new_test_cols =  feather.read_dataframe('gropped_columns_per_month_to_last_ver2')
train = pd.merge(train,new_test_cols,on='card_id',how='left')
test = pd.merge(test,new_test_cols,on='card_id',how='left')

train_out = feather.read_dataframe('train_months_outliner_percent')
test_out = feather.read_dataframe('test_months_outliner_percent')

train = pd.merge(train,train_out,on='card_id',how='left')
test = pd.merge(test,test_out,on='card_id',how='left')

In [22]:
train_agg_by_city_state_sub_out_per = feather.read_dataframe('train_agg_by_city_state_sub_out_per')
test_agg_by_city_state_sub_out_per = feather.read_dataframe('test_agg_by_city_state_sub_out_per')

In [23]:
#cats like and percent
cities_states_subs_percent = [[x for x in train_agg_by_city_state_sub_out_per if re.search('perct',x) or x=='card_id'],[x for x in train_agg_by_city_state_sub_out_per if not re.search('perct',x)]]

In [24]:
#state,city,  and percent
train = pd.merge(train,train_agg_by_city_state_sub_out_per,on='card_id',how='left')
test = pd.merge(test,test_agg_by_city_state_sub_out_per,on='card_id',how='left')

In [25]:
#state,city,  and percent
#train = pd.merge(train,train_agg_by_city_state_sub_out_per[cities_states_subs_percent[1]],on='card_id',how='left')
# = pd.merge(test,test_agg_by_city_state_sub_out_per[cities_states_subs_percent[1]],on='card_id',how='left')

In [26]:
#coded merch and merch_cats
train = pd.merge(train,feather.read_dataframe('merch_and_merch_cat_prct'),on='card_id',how='left')
test = pd.merge(test,feather.read_dataframe('merch_and_merch_cat_prct'),on='card_id',how='left')

In [27]:
#coded percents
train = pd.merge(train,feather.read_dataframe('all_transactions_coded_percent_out'),on='card_id',how='left')
test = pd.merge(test,feather.read_dataframe('all_transactions_coded_percent_out'),on='card_id',how='left')

In [28]:
best_res = 10
best_test = 0 
best_z = 0

In [29]:
list_of_best_cols = []
disct_of_best_folds = {x:[10,0,0,0] for x in range(5)}

In [31]:
#isbest = False
#for z_test in sorted(list(set(all_cols.importance.astype(np.int16)))):
#for z_test in len(all_cols):
#for z_test in range(15,149,3):
for z_test in range(1):
#    best_cols = all_cols.index[-z_test:]
#    best_cols = list(all_cols[all_cols.importance > z_test].index)
    
    if 'best_cols' in locals():
        features = [c for c in best_cols if c not in ['card_id', 'first_active_month','last_active_month']]
    else:
        features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]

    additional_cats= [x for x in features if re.search('[th]_cat1_[^s]',x)]
#    additional_cats+= [x for x in features if re.match('state_id_[m|a][^e]',x) or re.match('subsector_id_[m|a][^e]',x)]
    categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + additional_cats 
    categorical_feats = list(set(categorical_feats))
    #categorical_feats += [x for x in ['last4_months_cat','first_active_month_coded','client_wave','during_promo'] if x in features]
    categorical_feats += [x for x in ['last4_months_cat','client_wave','during_promo'] if x in features]
    #categorical_feats = list(set(categorical_feats)) + [x for x in ['client_wave','during_promo'] if x in features]
    
    warnings.filterwarnings("ignore")


#     for x_test in range(1):
#         if x_test == 0:
#             x_test+= 0.01
            
    for keys in range(4): 
        features,categorical_feats = best_disct_of_best_folds[keys][1],best_disct_of_best_folds[keys][3]
        
        param = {
                'num_leaves': 111, #checked
                 'num_threads': 4,
                 'max_bin': 170, #checked 170
                 'min_data_in_leaf': 100, #checked 100
                 'objective':'regression',
                 'max_depth': 9, #checked
                 'tree_learner':'data',
                 'learning_rate': 0.07,
                 "boosting": "gbdt", #dart, gbdt
                 "feature_fraction": 0.58, #checked 0.58
                 "bagging_freq": 7, #5
                 "bagging_fraction": 0.97, #checked 0.97
                 "bagging_seed": 11,
                 "metric": 'rmse',
                 "lambda_l1": 23, #checked 26 / 23
        #            'lambda_l2': 0.3, #checked
                 "random_state": 77,
                 "verbosity": -1,
                 "device":"gpu",
                 'gpu_device_id':0,
                 'gpu_platform_id':1,
                 'gpu_use_dp':True,
                }


        folds = KFold(n_splits=5, shuffle=True, random_state=15)
        oof = np.zeros(len(train))
        predictions = np.zeros(len(test))
        start = time.time()
        feature_importance_df = pd.DataFrame()

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
            print("fold n°{}".format(fold_))

            trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                                   label=target.iloc[trn_idx],
                                   categorical_feature=categorical_feats
                                  )
            val_data = lgb.Dataset(train.iloc[val_idx][features],
                                   label=target.iloc[val_idx],
                                   categorical_feature=categorical_feats
                                  )


            clf = lgb.train(param, 
                            trn_data,10000, 
                            valid_sets = [trn_data, val_data], 
                            verbose_eval=250, early_stopping_rounds = 250)

            oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
            
            #best fold score 
            this_fold_score = (mean_squared_error(oof[val_idx],target.iloc[val_idx])**0.5)
            if disct_of_best_folds[fold_][0] > this_fold_score:
                disct_of_best_folds[fold_][0] = this_fold_score
                disct_of_best_folds[fold_][1] = features
                disct_of_best_folds[fold_][2] = param
                disct_of_best_folds[fold_][3] = categorical_feats
                print("New best {}th fold score: {:<8.5f}\n".format(fold_,this_fold_score))
            
            fold_importance_df = pd.DataFrame()
            fold_importance_df["feature"] = features
            fold_importance_df["importance"] = clf.feature_importance()
            fold_importance_df["fold"] = fold_ + 1
            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

            predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
            
        
        errr = mean_squared_error(oof, target)**0.5

        if errr < best_res:
            best_res = errr
            best_test = x_test
            best_z = z_test
            print(f'better than ever {errr}')
#            isbest = True


     #   print(f"x_test = {x_test}")
        print("CV score: {:<8.5f}\n".format(errr))
    print(z_test)
    
    
#    if isbest:
#        break

fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.94054	valid_1's rmse: 3.66927
Early stopping, best iteration is:
[84]	training's rmse: 3.29095	valid_1's rmse: 3.6588
New best 0th fold score: 3.65880 

fold n°1
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.96156	valid_1's rmse: 3.62744
Early stopping, best iteration is:
[63]	training's rmse: 3.37667	valid_1's rmse: 3.61581
New best 1th fold score: 3.61581 

fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.94532	valid_1's rmse: 3.60401
Early stopping, best iteration is:
[100]	training's rmse: 3.26798	valid_1's rmse: 3.59086
New best 2th fold score: 3.59086 

fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.90547	valid_1's rmse: 3.80491
Early stopping, best iteration is:
[68]	training's rmse: 3.31098	valid_1's rmse: 3.79287
New best 3th fold score: 3.7928

In [17]:
best_res,best_test,best_z #(3.648730325988364, 7) #(3.651, 0)

(3.649211432617431, 0.01, 19)

In [21]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

In [22]:
all_cols = pd.DataFrame(best_features.groupby(['feature'])['importance'].mean()).sort_values(by='importance')
all_cols

,importance
feature,
mi_merchant_category_id_percent_to_merchid_std,18.2
first_catched_promo,18.8
month_lag_mean,20.6
subsector_id_max,22.4
all_month_pos__6_prc,22.6
4to5_rel_mean,23.0
all_months_distance_between_purchases_skew,23.8
hist_month_lag_mean,23.8
3to4_rel_mean,24.0


In [18]:
best_cols = all_cols[all_cols.importance >19].index

In [19]:
best_cols

Index(['new_purchases_per_month_mean', 'all_purchases_mean_percent',
       'category_combined_with_cat1_modeplus', 'all_month_fetb__1',
       'proc_purchase_amount_max_mean', 'new_purchase_amount_std',
       'new_purchase_date_ptp', 'new_purchase_date_max',
       'agged_all_purchase_date_day_max', 'hist_month_diff_mean',
       'agged_non_merchant_category_id_modeplus', 'new_purchase_month_mean',
       'new_purchase_amount_max', 'auth_purchase_date_max',
       'category_combined_without_cat1_bottom_quarter_quantile',
       'agged_au_purchase_date_day_max', 'last_payments_day',
       'auth_month_diff_mean',
       'category_combined_with_cat1_bottom_quarter_quantile',
       'first_active_month_coded'],
      dtype='object', name='feature')

In [41]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submit_with_all_additions16.csv", index=False)

In [20]:
#FULL TABLE TEST
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
additional_cats= [x for x in features if re.search('[th]_cat1_[^s]',x)]
categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + additional_cats 
categorical_feats = list(set(categorical_feats)) + [x for x in ['client_wave','during_promo'] if x in features]
warnings.filterwarnings("ignore")

param = {
        'num_leaves': 111, #checked
         'num_threads': 4,
         'max_bin': 170, #checked 170
         'min_data_in_leaf': 100, #checked 100
         'objective':'regression',
         'max_depth': 9, #checked
         'tree_learner':'data',
         'learning_rate': 0.07,
         "boosting": "gbdt", #dart, gbdt
         "feature_fraction": 0.58, #checked 0.58
         "bagging_freq": 7, #5
         "bagging_fraction": 0.97, #checked
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.23, #checked 26
#            'lambda_l2': 0.3, #checked
         "random_state": 77,
         "verbosity": -1,
         "device":"gpu",
         'gpu_device_id':0,
         'gpu_platform_id':1,
         'gpu_use_dp':True,
        }


folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))

    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )

    clf = lgb.train(param, 
                    trn_data,10000, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval=250, early_stopping_rounds = 250)

    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

errr = mean_squared_error(oof, target)**0.5
print("CV score: {:<8.5f}\n".format(errr))


fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.96997	valid_1's rmse: 3.6865
Early stopping, best iteration is:
[66]	training's rmse: 3.33915	valid_1's rmse: 3.67566
fold n°1
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.99227	valid_1's rmse: 3.62827
Early stopping, best iteration is:
[69]	training's rmse: 3.34625	valid_1's rmse: 3.61458
fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 3.00017	valid_1's rmse: 3.59804
Early stopping, best iteration is:
[84]	training's rmse: 3.31718	valid_1's rmse: 3.5913
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.9426	valid_1's rmse: 3.80068
Early stopping, best iteration is:
[69]	training's rmse: 3.30196	valid_1's rmse: 3.78649
fold n°4
Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 2.96212	valid_1's rmse: 3.62826
Early stoppi

In [17]:
len(all_cols) - best_z,best_res

(35, 3.6515577082624713)

In [24]:
best_cols = disct_of_best_folds[0][1]

In [36]:
print(best_disct_of_best_folds[1][3])

['feature_1', 'agged_non_city_id_modeplus', 'agged_au_city_id_modeplus', 'category_combined_without_cat1_bottom_quarter_quantile', 'agged_non_merchant_category_id_modeplus', 'agged_all_merchant_category_id_modeplus', 'category_combined_with_cat1_modeplus', 'category_combined_without_cat1_top_quarter_quantile', 'category_combined_with_cat1_bottom_quarter_quantile', 'category_combined_with_cat1_top_quarter_quantile', 'agged_au_merchant_category_id_modeplus', 'category_combined_without_cat1_modeplus', 'agged_all_city_id_modeplus', 'last4_months_cat']


In [17]:
best_disct_of_best_folds[x][0]


NameError: name 'best_disct_of_best_folds' is not defined

In [ ]:
disct_of_best_folds

In [2]:
all_features = [c for c in best_cols if c not in ['card_id', 'first_active_month','last_active_month']]

for z_test in [c for c in best_cols if c not in ['card_id', 'first_active_month','last_active_month']]:
        
    features = [c for c in all_features if c not in ['card_id', 'first_active_month','last_active_month',z_test]]

    additional_cats= [x for x in features if re.search('[th]_cat1_[^s]',x)]
    categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + additional_cats 
    categorical_feats = list(set(categorical_feats))
    categorical_feats += [x for x in ['last4_months_cat','client_wave','during_promo'] if x in features]
    
    warnings.filterwarnings("ignore")


    for x_test in range(1):    
        param = {
                'num_leaves': 111, #checked
                 'num_threads': 4,
                 'max_bin': 170, #checked 170
                 'min_data_in_leaf': 100, #checked 100
                 'objective':'regression',
                 'max_depth': 9, #checked
                 'tree_learner':'data',
                 'learning_rate': 0.07,
                 "boosting": "gbdt", #dart, gbdt
                 "feature_fraction": 0.58, #checked 0.58
                 "bagging_freq": 7, #5
                 "bagging_fraction": 0.97, #checked 0.97
                 "bagging_seed": 11,
                 "metric": 'rmse',
                 "lambda_l1": 23, #checked 26 / 23
        #            'lambda_l2': 0.3, #checked
                 "random_state": 77,
                 "verbosity": -1,
                 "device":"gpu",
                 'gpu_device_id':0,
                 'gpu_platform_id':1,
                 'gpu_use_dp':True,
                }


        folds = KFold(n_splits=5, shuffle=True, random_state=15)
        oof = np.zeros(len(train))
        predictions = np.zeros(len(test))
        start = time.time()
        feature_importance_df = pd.DataFrame()

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
            print("fold n°{}".format(fold_))

            trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                                   label=target.iloc[trn_idx],
                                   categorical_feature=categorical_feats
                                  )
            val_data = lgb.Dataset(train.iloc[val_idx][features],
                                   label=target.iloc[val_idx],
                                   categorical_feature=categorical_feats
                                  )


            clf = lgb.train(param, 
                            trn_data,10000, 
                            valid_sets = [trn_data, val_data], 
                            verbose_eval=250, early_stopping_rounds = 250)

            oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
            
            #best fold score 
            this_fold_score = (mean_squared_error(oof[val_idx],target.iloc[val_idx])**0.5)
            if disct_of_best_folds[fold_][0] > this_fold_score:
                disct_of_best_folds[fold_][0] = this_fold_score
                disct_of_best_folds[fold_][1] = features
                disct_of_best_folds[fold_][2] = param
                disct_of_best_folds[fold_][3] = categorical_feats
                print("New best {}th fold score: {:<8.5f}\n".format(fold_,this_fold_score))
            
            predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
            
        
        errr = mean_squared_error(oof, target)**0.5

        if errr < best_res:
            best_res = errr
            best_test = x_test
            print(f'better than ever {errr}')
            print('%s is out' % z_test)
            all_features.remove(z_test)
            

        print("CV score: {:<8.5f}\n".format(errr))
    print(z_test)

NameError: name 'best_cols' is not defined

In [65]:
best_col_numbers = [1,2,3,4,5,6,7,9,10,11,12,13,14,15,17,18,19,22,23,24,25,26,27,28,29,32,33,44,50,54,58,106,123]

In [66]:
best_cols = [all_cols.index[-x] for x in best_col_numbers]

In [8]:
for x in disct_of_best_folds.keys():
    if disct_of_best_folds[x][0] > best_disct_of_best_folds[x][0]:
        disct_of_best_folds[x] = best_disct_of_best_folds[x]

NameError: name 'disct_of_best_folds' is not defined

In [80]:
import pickle
with open('list_of_best_folds.pickle', 'wb') as f:
    pickle.dump(disct_of_best_folds, f, pickle.HIGHEST_PROTOCOL)

In [81]:
with open('list_of_best_cols.pickle', 'wb') as f:
    pickle.dump(list_of_best_cols, f, pickle.HIGHEST_PROTOCOL)

In [30]:
import pickle
with open('list_of_best_folds.pickle', 'rb') as f:
    best_disct_of_best_folds = pickle.load(f)
with open('list_of_best_cols.pickle', 'rb') as f:
    best_list_of_best_cols = pickle.load(f)

In [14]:
best_list_of_best_cols

[0, 3, 20, 0, 27, 33, 35]

In [15]:
disct_of_best_folds = best_disct_of_best_folds

In [33]:
best_disct_of_best_folds[0][0]

3.660326406289772

In [69]:
target = pd.read_csv('target2',header=None,index_col=0)

In [79]:
target[1] = pd.cut(np.squeeze(target.values),bins=[-35,-19,-18]+[x-17 for x in range(0,36)],labels=[x for x in range(0,38)],include_lowest=True).astype(np.int16)

In [87]:
#isbest = False
for z_test in range(1):
    if 'best_cols' in locals():
        features = [c for c in best_cols if c not in ['card_id', 'first_active_month','last_active_month']]
    else:
        features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]

    additional_cats= [x for x in features if re.search('[th]_cat1_[^s]',x)]
#    additional_cats+= [x for x in features if re.match('state_id_[m|a][^e]',x) or re.match('subsector_id_[m|a][^e]',x)]
    categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + additional_cats


    warnings.filterwarnings("ignore")


    for x_test in range(1):
        if x_test == 0:
            x_test+= 0.01
        param = {
                'num_leaves': 111, #checked
                 'num_threads': 4,
                 'max_bin': 170, #checked 170
                 'min_data_in_leaf': 100, #checked 100
                 'objective':'softmax',
                 'num_class':38,
                 'max_depth': 9, #checked
                 'tree_learner':'data',
                 'learning_rate': 0.007,
                 "boosting": "gbdt", #dart, gbdt
                 "feature_fraction": 0.58, #checked 0.58
                 "bagging_freq": 7, #5
                 "bagging_fraction": 0.97, #checked
                 "bagging_seed": 11,
                 "metric": 'multi_logloss',
                 "lambda_l1": 0.23, #checked 26
    #            'lambda_l2': 0.3, #checked
                 "random_state": 77,
                 "verbosity": -1,
                 "device":"gpu",
                 'gpu_device_id':0,
                 'gpu_platform_id':1,
                 'gpu_use_dp':True,
                }


        folds = KFold(n_splits=5, shuffle=True, random_state=15)
        oof = np.zeros((len(train),38))
        predictions = np.zeros((len(test),38))
        start = time.time()
        feature_importance_df = pd.DataFrame()

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
            print("fold n°{}".format(fold_))

            trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                                   label=target.iloc[trn_idx],
                                   categorical_feature=categorical_feats
                                  )
            val_data = lgb.Dataset(train.iloc[val_idx][features],
                                   label=target.iloc[val_idx],
                                   categorical_feature=categorical_feats
                                  )


            clf = lgb.train(param, 
                            trn_data,30000, 
                            valid_sets = [trn_data, val_data], 
                            verbose_eval=250, early_stopping_rounds = 250)

            oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

            fold_importance_df = pd.DataFrame()
            fold_importance_df["feature"] = features
            fold_importance_df["importance"] = clf.feature_importance()
            fold_importance_df["fold"] = fold_ + 1
            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

            predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
 #       errr = mean_squared_error(oof, target)**0.5

 #       if errr < best_res:
 #           best_res = errr
 #           best_test = x_test
 #           best_z = z_test
 #           print(f'better than ever {errr}')
#            isbest = True


     #   print(f"x_test = {x_test}")
#        print("CV score: {:<8.5f}\n".format(errr))
#    print(z_test)
#    best_cols = all_cols[all_cols.importance > z_test].index
#     if isbest:
#         break

fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's multi_logloss: 1.65247	valid_1's multi_logloss: 1.80319
[500]	training's multi_logloss: 1.53486	valid_1's multi_logloss: 1.77599
[750]	training's multi_logloss: 1.45189	valid_1's multi_logloss: 1.76798
[1000]	training's multi_logloss: 1.38706	valid_1's multi_logloss: 1.76639
Early stopping, best iteration is:
[977]	training's multi_logloss: 1.3926	valid_1's multi_logloss: 1.76637
fold n°1
Training until validation scores don't improve for 250 rounds.
[250]	training's multi_logloss: 1.6525	valid_1's multi_logloss: 1.80744
[500]	training's multi_logloss: 1.53501	valid_1's multi_logloss: 1.78034
[750]	training's multi_logloss: 1.45188	valid_1's multi_logloss: 1.77258
[1000]	training's multi_logloss: 1.38773	valid_1's multi_logloss: 1.77119
Early stopping, best iteration is:
[981]	training's multi_logloss: 1.39219	valid_1's multi_logloss: 1.77117
fold n°2
Training until validation scores don't improve 

In [98]:
max_vals = np.max(predictions,axis=1)

In [99]:
copy_preds = predictions

In [102]:
max_3_values = np.zeros(copy_preds.shape)

In [ ]:
first_max = copy_preds.T == max_vals

In [103]:
max_3_values += first_max.T

In [107]:
copy_preds = max_3_values * -2 + copy_preds

In [ ]:
first_max = copy_preds.T == max_vals
max_3_values += first_max.T

In [113]:
def find_N_max_values(matrix,N):
    matrix_max_values = np.zeros(matrix.shape)
    for x in range(N):
        max_this_step = np.max(matrix,axis=1)
        max_this_step = (matrix.T == max_this_step).T
        matrix_max_values+= max_this_step
        matrix += matrix_max_values * -2
    return matrix_max_values

In [ ]:
actual_values = np.array([-33,-18.5]+[x-17.5 for x in range(0,36)])

In [141]:
max_vals = find_N_max_values(copy_preds,2)

In [144]:
score = (actual_values * max_vals).mean(axis=1)

In [15]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("shop_overfit.csv", index=False)

In [147]:
best_score = pd.read_csv('submit_with_all_additions16.csv')

In [156]:
xgboost =  pd.read_csv('submit_with_all_additionsXGBOOST.csv')

In [159]:
score_combo = (best_score.target * 0.96) + (sub_df.target * 0.4)

In [160]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = score_combo
sub_df.to_csv("submit1_softmax_and_best.csv", index=False)